In [62]:
#necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as random

setupdirectory = "./Setup-Files/"

In [63]:
#defining the grid
cords = []
map_size = 12
for i in range(map_size):
    for j in range(map_size):
        cords.append((i, j))


#defining location of properties
number_of_properties = 10
propcords = []
for i in range(number_of_properties):
    propcords.append((rd.randint(0, map_size-1), rd.randint(0, map_size-1)))
print(propcords)

[(7, 4), (8, 9), (10, 0), (4, 4), (3, 9), (10, 5), (2, 6), (4, 0), (7, 11), (11, 2)]


In [64]:
#getting prop cords and values into a dataframe
props_1_2_df = pd.read_csv(setupdirectory + "Buildings_FL_E_1.2_2026.txt", sep="\s+", header=None, engine='python')
print(props_1_2_df)
props_1_2_df = props_1_2_df.rename(columns={0: "Type", 1: "x", 2: "y", 3: "value"})
print(props_1_2_df)
props_1_2_df["Cords"] = list(zip(props_1_2_df.x, props_1_2_df.y))
print(props_1_2_df)
props_1_3_df = pd.read_csv(setupdirectory + "Buildings_FL_F_1.3_2026.txt", sep="\s+", header=None, engine='python')
props_1_3_df = props_1_3_df.rename(columns={0: "Type", 1: "x", 2: "y", 3: "value"})
props_1_3_df["Cords"] = list(zip(props_1_3_df.x, props_1_3_df.y))
print(props_1_3_df)

       0   1   2   3
0      H   5  10  27
1      H   6  10  33
2      H   2   9  30
3      H   3   6  24
4      H   7   6  21
5      H  10   6  30
6      H   7   4  27
7      H   6   3  30
8      H   7   2  30
9      H   8   1  30
10  Hot.   4   5  36
11  Bank   6   5  39
12  Shop  11   2  30
13    Dr   3   8  30
14  P.O.   8   5  33
    Type   x   y  value
0      H   5  10     27
1      H   6  10     33
2      H   2   9     30
3      H   3   6     24
4      H   7   6     21
5      H  10   6     30
6      H   7   4     27
7      H   6   3     30
8      H   7   2     30
9      H   8   1     30
10  Hot.   4   5     36
11  Bank   6   5     39
12  Shop  11   2     30
13    Dr   3   8     30
14  P.O.   8   5     33
    Type   x   y  value    Cords
0      H   5  10     27  (5, 10)
1      H   6  10     33  (6, 10)
2      H   2   9     30   (2, 9)
3      H   3   6     24   (3, 6)
4      H   7   6     21   (7, 6)
5      H  10   6     30  (10, 6)
6      H   7   4     27   (7, 4)
7      H   6   3

In [65]:
def get_scs_chords():
    #propertycoordinates work like the following: the property with the coordinates (1,1) is the bottom left property, thus it covers [0,1]x[0,1], the property with the coordinates (2,1) is the property to the right of it and covers [1,2]x[0,1].
    #stormcenter coordinates are on edges of properties, thus the stormcenter with coordinates (1,1) is on the edge of the property with coordinates (1,1) and (2,1), the stormcenter with coordinates (0,0) is on the edge of the property with coordinates (0,0) and (1,0) and so on.

    scs_cords = []
    x_cord = random.randint(-4, 16)
    y_cord = random.randint(-4, 16)
    #print(f"Storm Center Cords: ({x_cord}, {y_cord})")
    dice1 = random.randint(1, 6)
    dice2 = random.randint(1, 6)
    sum_dice = dice1 + dice2
    #print(f"Dice 1: {dice1}, Dice 2: {dice2}, Sum: {sum_dice}")
    if sum_dice <= 6:
        pass
    elif sum_dice == 7:
        for i in range(0, 2):
            for j in range(0, 2):
                scs_cords.append((x_cord + i, y_cord + j))
    elif sum_dice == 8 or sum_dice == 9:
        for i in range(-1, 3):
            for j in range(-1, 3):
                scs_cords.append((x_cord + i, y_cord + j))
    elif sum_dice == 10 or sum_dice == 11:
        for i in range(-2, 4):
            for j in range(-2, 4):
                scs_cords.append((x_cord + i, y_cord + j))
    elif sum_dice == 12:
        for i in range(-4, 6):
            for j in range(-4, 6):
                scs_cords.append((x_cord + i, y_cord + j))
    #print(len(scs_cords))
    return scs_cords

print(get_scs_chords())




[]


In [66]:
def calculate_damage(scs_cords, prop_df):
    damage = 0
    types = []
    for index, row in prop_df.iterrows():
        if row["Cords"] in scs_cords:
            damage += row["value"]
            types.append(row["Type"])
    
    return damage, types

In [67]:
losses_1_2 = []
losses_1_3 = []
types_1_2_all = []
types_1_3_all = []
for i in range(100):
    scs_cords = get_scs_chords()
    damage_1_2, types_1_2 = calculate_damage(scs_cords, props_1_2_df)
    damage_1_3, types_1_3 = calculate_damage(scs_cords, props_1_3_df)
    losses_1_2.append(damage_1_2)
    losses_1_3.append(damage_1_3)
    types_1_2_all.extend(types_1_2)
    types_1_3_all.extend(types_1_3)

print(f"Losses for 1.2: {losses_1_2}")
print(f"Losses for 1.3: {losses_1_3}")


Losses for 1.2: [0, 0, 0, 273, 0, 0, 0, 0, 117, 0, 0, 0, 0, 0, 0, 183, 0, 87, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 129, 189, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 30, 0, 54, 0, 0, 0, 0, 210, 0, 0, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 189, 66, 0, 0, 0]
Losses for 1.3: [0, 0, 0, 246, 0, 0, 0, 0, 93, 0, 0, 0, 0, 0, 0, 186, 0, 63, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 69, 150, 27, 30, 0, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 117, 60, 0, 117, 0, 63, 0, 0, 117, 63, 0, 30, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 132, 39, 0, 0, 0]
